In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt

In [2]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, [0, 1, -1]])
    for i in range(len(data)):
        if data[i,-1] == 0:
            data[i,-1] = -1
    # print(data)
    return data[:,:2], data[:,-1]

In [3]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [131]:
class Adaboost:
    def __init__(self,learning_rate=0.5):
        self.learning_rate=learning_rate
        
    def _em(self,axis,direct,threshold):
        if direct=='p':
            tmpl=[1 if i > threshold else -1 for i in self.x[:,axis] ]
        else:
            tmpl=[-1 if i > threshold else 1 for i in self.x[:,axis] ]
        print('tmpl: '+str(tmpl))
        tmpl2=self.y-tmpl
        tmpl3=[0 if i ==0 else 1 for i in tmpl2]
        print('tmpl3: '+str(tmpl3))
        print('D'+str(self.D))
        error=np.sum(np.multiply(tmpl3,self.D))
        print('error: '+str(error))
        return error
    
    def _predict(self,axis,threshold,direct):
        if direct=='p':
            tmpl=[1 if i > threshold else -1 for i in self.x[:,axis] ]
        else:
            tmpl=[-1 if i > threshold else 1 for i in self.x[:,axis] ]
        return tmpl
        
            
    def fit(self,x,y):
        self.x=np.array(x)
        self.y=np.array(y)
        self.m,self.n=self.x.shape
        #self.x.shape
        
        self.funList=list()
        self.D=np.ones(self.m)/float(self.m)
        
        minError=np.inf
        minTree=None
        while minError>0.01:
            for axis in range(self.n):
                lmin=np.min(self.x[:,axis])
                lmax=np.max(self.x[:,axis])
                for threshold in np.arange(lmin,lmax,self.learning_rate):
                    print('threshold:'+str(threshold))
                    e1=self._em(axis,'p',threshold)
                    if minError>e1:
                        minError=e1
                        minTree=(axis,threshold,'p',e1) 
                    e2=self._em(axis,'n',threshold)
                    if minError>e2:
                        minError=e2
                        minTree=(axis,threshold,'n',e2) 
#                     print('e1: '+str(e1))
#                     print('e2: '+str(e2))
#                     e=e1 if e1<e2 else e2
#                     d='p' if e1<e2 else 'n'
#                     if minError>e:
#                         minError=e
#                         minTree=(axis,threshold,d,e)
#                     print('minError'+str(minError))
#                     print(minTree)
            alpha=0.5*np.log((1-minError)/float(minError))
            self.funList.append((alpha,minTree))
            yi=self.y
            gm=np.array(self._predict(minTree[0],minTree[1],minTree[2]))
            print('alpha: '+str(alpha))
            print('yi: '+str(yi))
            print('gm: '+str(gm))
            eta=np.multiply(-alpha*yi,gm)
            Z=sum(np.multiply(self.D,np.exp(eta)))
            print('e eta:' +str(np.exp(eta)))
            self.D=np.multiply(self.D,np.exp(eta))/Z

    def predict(self,x):
        sum=0
        print(self.funList)
        for fun in self.funList:
            alpha=fun[0]
            tree=fun[1]
#             print(tree)
#             print(tree[0])
            sum+=np.sum(alpha*np.array(self._predict(tree[0],tree[1],tree[2])))
#         print(sum)
        return 1 if sum>0 else -1

In [132]:
dataSet = [[0, 1, 3], [0, 3, 1], [1, 2, 2], [1, 1, 3], [1, 2, 3], [0, 1, 2], [1, 1, 2], [1, 1, 1], [1, 3, 1], [0, 2, 1]]    #p153的例子
labels = [-1, -1, -1, -1, -1, -1, 1, 1, -1, -1]
adaboost = Adaboost()
adaboost.fit(dataSet,labels)

threshold:0.0
tmpl: [-1, -1, 1, 1, 1, -1, 1, 1, 1, -1]
tmpl3: [0, 0, 1, 1, 1, 0, 0, 0, 1, 0]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
error: 0.4
tmpl: [1, 1, -1, -1, -1, 1, -1, -1, -1, 1]
tmpl3: [1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
error: 0.6
threshold:0.5
tmpl: [-1, -1, 1, 1, 1, -1, 1, 1, 1, -1]
tmpl3: [0, 0, 1, 1, 1, 0, 0, 0, 1, 0]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
error: 0.4
tmpl: [1, 1, -1, -1, -1, 1, -1, -1, -1, 1]
tmpl3: [1, 1, 0, 0, 0, 1, 1, 1, 0, 1]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
error: 0.6
threshold:1.0
tmpl: [-1, 1, 1, -1, 1, -1, -1, -1, 1, 1]
tmpl3: [0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
error: 0.7
tmpl: [1, -1, -1, 1, -1, 1, 1, 1, -1, -1]
tmpl3: [1, 0, 0, 1, 0, 1, 0, 0, 0, 0]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
error: 0.30000000000000004
threshold:1.5
tmpl: [-1, 1, 1, -1, 1, -1, -1, -1, 1, 1]
tmpl3: [0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
D[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
e

tmpl3: [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
D[1.66666667e-01 1.64995661e-01 1.43899347e-05 1.66666667e-01
 1.43899347e-05 1.66666667e-01 1.64995661e-01 4.45078437e-03
 1.64995661e-01 5.33451319e-04]
error: 0.5044795642422948
tmpl: [-1, 1, -1, -1, -1, -1, -1, 1, 1, 1]
tmpl3: [0, 1, 0, 0, 0, 0, 1, 0, 1, 1]
D[1.66666667e-01 1.64995661e-01 1.43899347e-05 1.66666667e-01
 1.43899347e-05 1.66666667e-01 1.64995661e-01 4.45078437e-03
 1.64995661e-01 5.33451319e-04]
error: 0.4955204357577055
threshold:2.0
tmpl: [1, -1, -1, 1, 1, -1, -1, -1, -1, -1]
tmpl3: [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
D[1.66666667e-01 1.64995661e-01 1.43899347e-05 1.66666667e-01
 1.43899347e-05 1.66666667e-01 1.64995661e-01 4.45078437e-03
 1.64995661e-01 5.33451319e-04]
error: 0.5027941691203662
tmpl: [-1, 1, 1, -1, -1, 1, 1, 1, 1, 1]
tmpl3: [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
D[1.66666667e-01 1.64995661e-01 1.43899347e-05 1.66666667e-01
 1.43899347e-05 1.66666667e-01 1.64995661e-01 4.45078437e-03
 1.64995661e-01 5.33451319e-04]
error: 0.

error: 0.6666667446500182
threshold:1.0
tmpl: [-1, 1, 1, -1, 1, -1, -1, -1, 1, 1]
tmpl3: [0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
D[1.66666667e-01 1.66666589e-01 3.94139165e-14 1.66666667e-01
 3.94139165e-14 1.66666667e-01 1.66666589e-01 3.26294338e-08
 1.66666589e-01 2.01320779e-07]
error: 0.49999999999999994
tmpl: [1, -1, -1, 1, -1, 1, 1, 1, -1, -1]
tmpl3: [1, 0, 0, 1, 0, 1, 0, 0, 0, 0]
D[1.66666667e-01 1.66666589e-01 3.94139165e-14 1.66666667e-01
 3.94139165e-14 1.66666667e-01 1.66666589e-01 3.26294338e-08
 1.66666589e-01 2.01320779e-07]
error: 0.5
threshold:1.5
tmpl: [-1, 1, 1, -1, 1, -1, -1, -1, 1, 1]
tmpl3: [0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
D[1.66666667e-01 1.66666589e-01 3.94139165e-14 1.66666667e-01
 3.94139165e-14 1.66666667e-01 1.66666589e-01 3.26294338e-08
 1.66666589e-01 2.01320779e-07]
error: 0.49999999999999994
tmpl: [1, -1, -1, 1, -1, 1, 1, 1, -1, -1]
tmpl3: [1, 0, 0, 1, 0, 1, 0, 0, 0, 0]
D[1.66666667e-01 1.66666589e-01 3.94139165e-14 1.66666667e-01
 3.94139165e-14 1.66666667e-01 1.6

D[1.66666667e-01 1.66666667e-01 2.84951769e-25 1.66666667e-01
 2.84951769e-25 1.66666667e-01 1.66666667e-01 2.35323766e-13
 1.66666667e-01 2.01815406e-13]
error: 0.5000000000000223
tmpl: [-1, 1, 1, -1, -1, 1, 1, 1, 1, 1]
tmpl3: [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
D[1.66666667e-01 1.66666667e-01 2.84951769e-25 1.66666667e-01
 2.84951769e-25 1.66666667e-01 1.66666667e-01 2.35323766e-13
 1.66666667e-01 2.01815406e-13]
error: 0.4999999999999777
threshold:2.5
tmpl: [1, -1, -1, 1, 1, -1, -1, -1, -1, -1]
tmpl3: [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
D[1.66666667e-01 1.66666667e-01 2.84951769e-25 1.66666667e-01
 2.84951769e-25 1.66666667e-01 1.66666667e-01 2.35323766e-13
 1.66666667e-01 2.01815406e-13]
error: 0.5000000000000223
tmpl: [-1, 1, 1, -1, -1, 1, 1, 1, 1, 1]
tmpl3: [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
D[1.66666667e-01 1.66666667e-01 2.84951769e-25 1.66666667e-01
 2.84951769e-25 1.66666667e-01 1.66666667e-01 2.35323766e-13
 1.66666667e-01 2.01815406e-13]
error: 0.4999999999999777
alpha: 0.986646961044256

In [133]:
print(adaboost.predict([1, 3, 2]))

[(0.4236489301936017, (1, 1.0, 'n', 0.30000000000000004)), (0.45814536593707733, (1, 2.0, 'p', 0.28571428571428575)), (0.45814536593707733, (1, 2.0, 'p', 0.28571428571428575)), (0.6931471805599451, (1, 1.0, 'n', 0.20000000000000007)), (0.6931471805599451, (1, 1.0, 'n', 0.20000000000000007)), (0.9032097088193696, (2, 1.0, 'n', 0.14107142857142863)), (0.9032097088193696, (2, 1.0, 'n', 0.14107142857142863)), (0.9443492739848266, (1, 1.0, 'n', 0.13139293139293146)), (0.9443492739848266, (1, 1.0, 'n', 0.13139293139293146)), (0.9754312939556853, (1, 2.0, 'p', 0.12445933221423884)), (0.9754312939556853, (1, 2.0, 'p', 0.12445933221423884)), (0.9799692132726963, (1, 1.0, 'n', 0.12347371137973935)), (0.9799692132726963, (1, 1.0, 'n', 0.12347371137973935)), (0.9850995900172403, (2, 1.0, 'n', 0.12236749453501916)), (0.9850995900172403, (2, 1.0, 'n', 0.12236749453501916)), (0.9858180797263714, (1, 1.0, 'n', 0.12221325573591141)), (0.9858180797263714, (1, 1.0, 'n', 0.12221325573591141)), (0.98643211

In [118]:
def pred_score(obj,x,y):
    count=0
    l=len(y)
    for i in range(l):
        if y[i]==obj.predict(x[i]):
            count+=1
    return count/float(l)

In [134]:
adaboost2 = Adaboost(learning_rate=0.2)
adaboost2.fit(X_train, y_train)
pred_score(adaboost2,X_test,y_test)

threshold:4.3
tmpl: [1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tmpl3: [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0]
D[0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125
 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125
 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125
 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125
 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125
 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125
 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125 0.0125

error: 0.5637254901960774
tmpl: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
tmpl3: [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1]
D[0.00735294 0.00735294 0.00735294 0.00735294 0.00735294 0.04166667
 0.00735294 0.00735294 0.04166667 0.00735294 0.00735294 0.00735294
 0.00735294 0.00735294 0.00735294 0.00735294 0.00735294 0.00735294
 0.00735294 0.00735294 0.00735294 0.00735294 0.00735294 0.00735294
 0.00735294 0.00735294 0.00735294 0.04166667 0.00735294 0.00735294
 0.00735294 0.00735294 0.04166667 0.00735294 0.00735294 0.0073529

0.4